In [1]:
# 初期設定
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

import os, boto3, json, sagemaker
import numpy as np
from sagemaker.pytorch import PyTorchModel
from io import BytesIO

def make_dir(path):
    if os.path.isdir(path):
        pass
    else:
        os.mkdir(path)

sagemaker_session = sagemaker.Session()

# AWS設定
region = boto3.Session().region_name
role = 'YOUR_AWS_ROLE_NAME' # with s3, sagemaker accese policy
bucket='YOUR_BUCKET_NAME'
prefix = 'batch-images'
bucket_path = 'https://s3-{}.amazonaws.com/{}'.format(region,bucket)
input_prefix = os.path.join(prefix, 'inputs')
output_prefix = os.path.join(prefix, 'outputs')
inference_prefix = os.path.join(prefix, 'f_inference')

# Local設定
base_dir = os.getcwd()
data_dir = os.path.join(base_dir, 'data')
input_dir = os.path.join(base_dir, 'inputs')
output_dir = os.path.join(base_dir, 'outputs')
inference_dir = os.path.join(base_dir, 'inference')
source_dir = os.path.join(base_dir, 'src')
for dir_name in [input_dir, output_dir, source_dir, inference_dir]:
    make_dir(dir_name)

In [116]:
'''
Inference用のデータを作る(jsonlines)
    - Sampling
    - Convert Image into bytes
    - Save as jsonlines
'''
from src.utils import image_to_bytes

r_inference_path = os.path.join(base_dir, 'real_inference')
n = 1000
json_name = r_inference_path+f'/inf_data{n}.jsonl'

image_to_bytes(json_name, data_dir, inference_dir, n)

f_inference = sagemaker_session.upload_data(path=r_inference_path, bucket=bucket, key_prefix='batch-images/r_inference')

## Traning Model

In [4]:
# Prepare dataset for the model training

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
])

train_data = datasets.CIFAR10(root=data_dir, train=True, download=True, transform=transform)
test_data = datasets.CIFAR10(root=data_dir, train=False, download=True, transform=transform)

training_data_loader = DataLoader(train_data, batch_size=len(train_data))
training_data_loaded = next(iter(training_data_loader))
torch.save(training_data_loaded, os.path.join(input_dir, 'training.pt'))

test_data_loader = DataLoader(test_data, batch_size=len(test_data))
test_data_loaded = next(iter(test_data_loader))
torch.save(test_data_loaded, os.path.join(output_dir, 'test.pt'))

inputs = sagemaker_session.upload_data(path=input_dir, bucket=bucket, key_prefix=input_prefix)
outputs = sagemaker_session.upload_data(path=output_dir, bucket=bucket, key_prefix=output_prefix)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# Create Training Container
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point="resnet.py",
                    role=role,
                    source_dir = "src",
                    framework_version='1.12.0',
                    py_version='py38',
                    instance_count=1,
                    instance_type='ml.m4.xlarge',
                    hyperparameters={
                        'batch-size':128,
                        'lr': 0.01,
                        'epochs': 1,
                    })

estimator.fit({'training': inputs})

In [ ]:
cifar10_predictor = estimator.deploy(
    initial_instance_count=1, 
    instance_type='ml.m4.xlarge'
)

In [8]:
test_input = torch.rand(1,3,32,32)
test_output = cifar10_predictor.predict(test_input)
print(test_input.shape)
print(test_output.shape)

cifar10_predictor.delete_endpoint()

torch.Size([1, 3, 32, 32])
(1, 10)


# Batch Transformer

In [118]:

from sagemaker.pytorch.model import PyTorchModel

# model_path = cifar10_predictor
model_path = 's3://your/saved/model/path/model.tar.gz'

pytorch_model = PyTorchModel(model_data = model_path,
                             entry_point='resnet_batch_transform.py',
                             source_dir = 'src',
                             framework_version='1.12.0',
                             py_version='py38',
                             role = role)

In [124]:
max_concurrent_transforms = 10
max_payload = 1
# strategy = 'SingleRecord'
# split_type = None
strategy = 'MultiRecord'
split_type = 'Line'

output_s3_path = 's3://your/output/path'

transformer = pytorch_model.transformer(instance_count=1,
                              instance_type="ml.m5.xlarge",
                              max_concurrent_transforms=max_concurrent_transforms,
                              max_payload=max_payload,  
                              strategy=strategy,
                              output_path=output_s3_path)


In [ ]:

# データをセーブしておいたPath (f_inference)
inference_path = f's3://your/input/path'

transformer.transform(
    data=inference_path,
    data_type="S3Prefix",
    content_type="application/jsonlines",
    wait=True,
    split_type=split_type,
)